In [ ]:

!pip install --quiet openai
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "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"   # Groq API key
os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"

client = OpenAI()
# 2. Conversation Storage

conversation_history = []
run_counter = 0

def add_message(role, content):
    """Add user/assistant/system message to history"""
    conversation_history.append({"role": role, "content": content})

def print_history():
    """Print conversation history"""
    print("\n--- Conversation History ---")
    for m in conversation_history:
        print(f"{m['role']}: {m['content']}")
# 3. Summarization Function

def summarize_history(history):
    """Summarize entire conversation so far using Groq API"""
    messages = [{"role": m["role"], "content": m["content"]} for m in history]
    completion = client.chat.completions.create(
        model="moonshotai/kimi-k2-instruct-0905",   # Used Moonshotai model (Groq-supported model)
        messages=[
            {"role": "system", "content": "Summarize the following conversation concisely."},
            *messages
        ],
        max_tokens=100
    )
    return completion.choices[0].message.content.strip()
# 4. Truncation Options

def truncate_by_turns(n):
    """Keep only the last n conversation turns"""
    global conversation_history
    conversation_history = conversation_history[-n:]

def truncate_by_length(max_chars):
    """Keep only messages within max character length"""
    global conversation_history
    result, total_len = [], 0
    for m in reversed(conversation_history):
        if total_len + len(m["content"]) <= max_chars:
            result.insert(0, m)
            total_len += len(m["content"])
        else:
            break
    conversation_history = result
# 5. Periodic Summarization

def on_new_run(role, content, k=3):
    """
    Add message, increment run counter,
    summarize every k-th run, replace history with summary + last message
    """
    global run_counter, conversation_history

    add_message(role, content)
    run_counter += 1

    if run_counter % k == 0:
        print("\n[Summarizing conversation...]")
        summary = summarize_history(conversation_history)
        conversation_history = [{"role": "system", "content": f"SUMMARY -> {summary}"}]
        conversation_history.append({"role": role, "content": content})

    print_history()
# 6. Demonstration

print("=== Demo: Conversation Flow ===")

#Technical Project Discussion
on_new_run("user", "Hello! I'm Alex and I need assistance with my development task.")
on_new_run("assistant", "Hi Alex! I'm here to help. What kind of development task are you working on?")
on_new_run("user", "I'm building a system that creates summaries of chats using the Groq platform.")  # triggers summarization at 3rd run
on_new_run("assistant", "That sounds interesting! We'll need to handle conversation storage and create condensed versions.")
on_new_run("user", "Exactly, and we should also limit message length when conversations get lengthy.")

#AI Research Context
on_new_run("user", "Hi there! I'm Maya, working on an AI research assignment.")
on_new_run("assistant", "Hello Maya! I'd be glad to assist with your research. What's the focus area?")
on_new_run("user", "It involves implementing automatic text condensation using Groq's language models.")  # triggers summarization at 3rd run
on_new_run("assistant", "Excellent topic! This requires managing dialogue records and generating brief overviews.")
on_new_run("user", "Right, plus we need mechanisms to shorten content when it becomes excessive.")

# Academic Project Context
on_new_run("user", "Good day! I'm Sam, and I'm stuck on my coursework assignment.")
on_new_run("assistant", "Hi Sam! What subject is your assignment focused on?")
on_new_run("user", "It's about creating intelligent text compression systems with Groq API integration.")  # triggers summarization at 3rd run
on_new_run("assistant", "Interesting challenge! You'll need to track conversations and produce concise summaries.")
on_new_run("user", "Precisely, and implement features to control message volume when discussions expand.")


print("\n=== Demo: Truncation by last 2 turns ===")
truncate_by_turns(2)
print_history()

print("\n=== Demo: Truncation by 50 characters ===")
truncate_by_length(50)
print_history()



=== Demo: Conversation Flow ===

--- Conversation History ---
user: Hello! I'm Alex and I need assistance with my development task.

--- Conversation History ---
user: Hello! I'm Alex and I need assistance with my development task.
assistant: Hi Alex! I'm here to help. What kind of development task are you working on?

[Summarizing conversation...]

--- Conversation History ---
system: SUMMARY -> Got it—you're building a chat-summarization service on Groq. Let me know what part you need help with (API calls, prompt engineering, throughput, cost, etc.).
user: I'm building a system that creates summaries of chats using the Groq platform.

--- Conversation History ---
system: SUMMARY -> Got it—you're building a chat-summarization service on Groq. Let me know what part you need help with (API calls, prompt engineering, throughput, cost, etc.).
user: I'm building a system that creates summaries of chats using the Groq platform.
assistant: That sounds interesting! We'll need to handle conver

In [ ]:

from openai import OpenAI
import os, json
from jsonschema import validate, ValidationError

os.environ["OPENAI_API_KEY"] = "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"   # Groq API key
os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"

client = OpenAI()
# JSON Schema for extracting details

user_info_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}
def extract_user_info(conversation_text):
    response = client.chat.completions.create(
        model="moonshotai/kimi-k2-instruct-0905",   # Groq supported model
        messages=[
            {"role": "system", "content": "Extract user details into structured JSON."},
            {"role": "user", "content": conversation_text}
        ],
        functions=[
            {
                "name": "extract_user_info",
                "description": "Extracts personal details from conversation",
                "parameters": user_info_schema
            }
        ],
        function_call={"name": "extract_user_info"}
    )

    args = response.choices[0].message.function_call.arguments
    return json.loads(args)
def validate_extracted(data):
    try:
        validate(instance=data, schema=user_info_schema)
        return True, None
    except ValidationError as e:
        return False, str(e)
sample_chats = [
    "Hello, I'm Priya Patel, working as a software developer. You can contact me at priya.patel@techfirm.com, my phone is 8765432109, I'm based in Pune, and I'm 27 years old.",
"Good morning! This is Vikram Kumar, age 32. Feel free to reach out at vikram.k@startup.io or dial 9234567890. I'm currently residing in Hyderabad.",
"Hi there! I'm Sneha Reddy, 24 years old. My email address is sneha.reddy@company.com, contact number is 9876501234, and I live in Chennai.",
"Greetings! I'm Amit Gupta from the marketing team. My email is amit.gupta@business.com, phone number is 9123478056, I'm located in Noida, and I'm 29 years old.",
"Hi, this is Kavya Sharma speaking, I'm 26. You can reach me through kavya.s@consulting.com or call 8901234567. I work from Gurgaon.",
"Hello! My name is Rohan Joshi, I'm 31 years old. Contact details: rohan.joshi@finance.com, mobile: 9087654321, and I'm based in Mumbai."


]

for i, chat in enumerate(sample_chats, 1):
    print(f"\n=== Sample Chat {i} ===")
    extracted = extract_user_info(chat)
    is_valid, error = validate_extracted(extracted)

    print("Extracted JSON:", extracted)
    print("Validation:", "PASS ✅" if is_valid else f"FAIL ❌ - {error}")





=== Sample Chat 1 ===
Extracted JSON: {'age': 27, 'email': 'priya.patel@techfirm.com', 'location': 'Pune', 'name': 'Priya Patel', 'phone': '8765432109'}
Validation: PASS ✅

=== Sample Chat 2 ===
Extracted JSON: {'age': 32, 'email': 'vikram.k@startup.io', 'location': 'Hyderabad', 'name': 'Vikram Kumar', 'phone': '9234567890'}
Validation: PASS ✅

=== Sample Chat 3 ===
Extracted JSON: {'age': 24, 'email': 'sneha.reddy@company.com', 'location': 'Chennai', 'name': 'Sneha Reddy', 'phone': '9876501234'}
Validation: PASS ✅

=== Sample Chat 4 ===
Extracted JSON: {'age': 29, 'email': 'amit.gupta@business.com', 'location': 'Nice', 'name': 'Amit Gupta', 'phone': '9123478056'}
Validation: PASS ✅

=== Sample Chat 5 ===
Extracted JSON: {'age': 26, 'email': 'kavya.s@consulting.com', 'location': 'Gurgaon', 'name': 'Kavya Sharma', 'phone': '8901234567'}
Validation: PASS ✅

=== Sample Chat 6 ===
Extracted JSON: {'age': 31, 'email': 'rohan.joshi@finance.com', 'location': 'Mumbai', 'name': 'Rohan Joshi', 